In [2]:
from konlpy.tag import Twitter

import pandas as pd
import pickle
from math import log2,log10,sqrt

In [19]:
with open('artTokens.dat','rb') as file: # 미리 doc별로 저장한 tokens의 리스트를 load한다
    morphs = pickle.load(file)

In [3]:
with open('artNouns.dat','rb') as file: # 전처리 과정에서 저장한 doc별 nouns의 리스트를 load한다
    nouns = pickle.load(file)

In [6]:
with open('artAds.dat','rb') as file: # 전처리 과정에서 저장한 doc별 동사 형용사 부사 감탄사의 리스트를 load한다
    ads = pickle.load(file)

TF-IDF 기반으로 중요도 높은 단어를 뽑아보자

In [4]:
# %timeit set(n for n in nouns[0]) # unique한 element뽑는데 set이 더 빠름

13.8 µs ± 1.36 µs per loop (mean ± std. dev. of 7 runs, 100000 loops each)


In [5]:
# lst = []
# %timeit [lst.append(n) for n in nouns[0] if n not in lst]

88.8 µs ± 2.51 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)


In [7]:
tokens = ads #어떤 품사의 토큰을 사용할지 지정한다 ads는 [동사,형용사,부사,감탄사]

In [20]:
tokens = morphs # 모든 품사를 사용하는 옵션

In [21]:
unq_tokens = []
for lst in tokens:
    unq_tokens += lst
unq_tokens = set(unq_tokens)

In [22]:
len(unq_tokens)

133869

In [23]:
doc_tokens = {}
maxCount = {}

for i,lst in enumerate(tokens):
    maxCount[i] = 0
    tmp_tokens = {}
    
    for w in lst:
        if w in tmp_tokens.keys():
            tmp_tokens[w] += 1
        else:
            tmp_tokens[w] = 1

        if maxCount[i] < tmp_tokens[w]:
            maxCount[i] = tmp_tokens[w]
                    
    doc_tokens[i] = tmp_tokens


In [28]:
doc_tokens[0]

{"'": 2,
 '(': 3,
 ')': 3,
 ',': 5,
 '.': 11,
 '1': 2,
 '10': 1,
 '11': 1,
 '12': 8,
 '2013': 1,
 '2014': 3,
 '22': 1,
 '3': 1,
 '8': 1,
 '<': 1,
 '>': 1,
 'GMA': 2,
 'Nostalgia': 1,
 'kbc': 1,
 '~': 1,
 '‘': 4,
 '’': 2,
 '“': 1,
 '”': 1,
 '∼': 1,
 'ⓒ': 1,
 '가': 3,
 '각': 1,
 '간': 1,
 '감상할': 1,
 '개관': 1,
 '개최': 2,
 '갤러리': 2,
 '것': 3,
 '계기': 1,
 '고': 1,
 '고향': 2,
 '곳': 1,
 '공동': 1,
 '과': 3,
 '과정': 1,
 '관': 2,
 '관람객': 1,
 '관장': 1,
 '광주': 9,
 '구입': 1,
 '기회': 2,
 '기획': 2,
 '기획전': 1,
 '까지': 2,
 '남도': 6,
 '년': 4,
 '는': 3,
 '다': 3,
 '다를': 1,
 '다큐': 1,
 '달래는': 1,
 '대표': 1,
 '대표하는': 1,
 '대한': 1,
 '던': 1,
 '데': 1,
 '동': 1,
 '되었': 1,
 '된': 1,
 '될': 2,
 '들': 1,
 '라고': 1,
 '로': 2,
 '를': 12,
 '많은': 1,
 '명': 1,
 '모아': 1,
 '무대': 1,
 '문화': 4,
 '문화재단': 3,
 '물론': 2,
 '물씬': 1,
 '미술': 1,
 '미술관': 7,
 '바': 1,
 '반추': 1,
 '받은': 1,
 '발전': 1,
 '밝혔': 1,
 '방송': 1,
 '번째': 1,
 '봄': 1,
 '뿌리': 1,
 '사간': 1,
 '상록': 1,
 '서울': 1,
 '선정하여': 1,
 '성': 4,
 '성격': 1,
 '성장': 1,
 '소중한': 1,
 '수': 1,
 '시립': 6,
 '언론': 1,
 '에': 2,
 '에서

In [23]:
for i,(k,v) in enumerate(doc_tokens.items()):
    if i>10: break
    print("{0} - {1}/{2}".format(k, len(v), maxCount[k])) # 문서id-   등장단어수/가장 높았던 빈도

0 - 28/2
1 - 99/11
2 - 50/3
3 - 89/5
4 - 48/6
5 - 108/7
6 - 53/5
7 - 68/6
8 - 83/11
9 - 87/7
10 - 209/10


In [24]:
#시간이 좀 걸린다
invertedIdx = {}
for token in unq_tokens:
    invertedIdx[token] = []
    
    for k,v in doc_tokens.items():
        if token in v:
            invertedIdx[token].append(k)

In [30]:
# with open('invertedIdx_tokens.dic','wb') as file:
#     pickle.dump(invertedIdx, file)

In [ ]:
# with open('invertedIdx_nouns.dic','wb') as file:
#     pickle.dump(invertedIdx, file)

In [11]:
# with open('invertedIdx_ads.dic','wb') as file:
#     pickle.dump(invertedIdx, file)

In [31]:
with open('invertedIdx_tokens.dic','rb') as file: # 전체 형태소로 만든 invertedIdx
    invertedIdx = pickle.load(file)

In [16]:
with open('invertedIdx_nouns.dic','rb') as file: # 명사로 만든 invertedIdx
    invertedIdx = pickle.load(file)

In [4]:
with open('invertedIdx_ads.dic','rb') as file: #ads로 만든 invertedIdx
    invertedIdx = pickle.load(file)

In [32]:
K = 0.5 # K + (1-K)*(f(t,d)/maxf(t,d)) 최솟값 0.5, 최댓값 1 
TF = {}
for k,v in doc_tokens.items():
    tfList = {}
    
    for w in v:
        tfList[w] =  K + (1-K)*(v[w]/maxCount[k])
#         print('{0} | {1} + {2} *({3}/{4})  =  {5}'.format(
#             w, K, (1-K), v[w], maxCount[k], tfList[w]
#         ))
        
    TF[k] = tfList

In [33]:
docSize = len(tokens)
TFIDF = {}
"""
TF = doc1|{단어1:0.54, 단어2:056 ...} 
"""
for (k,v) in TF.items():
    idfList = {}
    for t in v:
        idf = log10(docSize/len(invertedIdx[t]))
        idfList[t] = v[t] * idf
#         print('{0} | {1} * log({2} / {3}) = {4}'.format(
#             t, v[t], docSize, len(invertedIdx[t]), idfList[t]
#         ))
    
    TFIDF[k] = idfList

In [38]:
with open('TFIDF.dic','wb') as file: #TFIDF 결과를 저장
    pickle.dump(TFIDF, file)

In [39]:
with open('TFIDF.dic','rb') as file:
    TFIDF = pickle.load(file)

In [40]:
len(TFIDF)

20514

In [37]:
TFIDF[1]

{'(': 0.12943291716419134,
 ')': 0.12363976094168501,
 '),': 0.5227651816788577,
 ',': 0.015120767996193092,
 '.': 0.001886262196434037,
 '1': 0.37477096320845316,
 '10': 0.491837437315793,
 '2': 0.3654145329791687,
 '2011': 0.6424894925043558,
 '2013': 0.673697214542074,
 '23': 0.982034869098158,
 '26': 0.9885536458710431,
 '28': 0.9616368387821178,
 '3': 0.3806793238224293,
 '32': 1.2572945109689846,
 '4': 0.4605934724814047,
 '8': 0.7335731180432384,
 '?': 0.3397890433421786,
 'Cre': 1.99072653455488,
 'Creative': 1.5783812201424696,
 'OCI': 1.5007974580530952,
 'Open': 1.4925931022602978,
 'Report': 1.8707709315667471,
 'Studio': 1.3825511313090382,
 'tive': 1.99072653455488,
 '‘': 0.14022802706769763,
 '’': 0.14129758310784993,
 'ⓒ': 0.004004550948435571,
 '展': 0.38963651099364643,
 '가': 0.022738589106779474,
 '가장': 0.3167923067627029,
 '가지': 0.2392978428742826,
 '가진': 0.41121474160565474,
 '갇혀': 0.8792106516206817,
 '감': 0.3786037494774371,
 '같은': 0.18582785299637658,
 '개최': 0.43

In [14]:
sortedByWeight = [sorted(dic.items(), key=lambda kv:kv[1],reverse=True) for dic in TFIDF.values()] # 내림차순 정렬해보자

In [16]:
# with open('sortedByWeight_ads.lst','wb') as file:
#     pickle.dump(sortedByWeight,file)

In [17]:
# with open('sortedByWeight_ads.lst','rb') as file:
#     sortedByWeight = pickle.load(file)

In [59]:
sortedByWeight[1][:20] #1번 doc의 TFIDF기반 중요단어 top10을 뽑았다

[('변화됨', 1.8315315500404983),
 ('붓고', 1.5310365670147266),
 ('살펴보기', 1.4450686469992713),
 ('마르', 1.3389370116812562),
 ('살아나는', 1.3261291412178802),
 ('희는', 1.2887238242919956),
 ('자라난', 1.2495605738783455),
 ('행복했', 1.242887234014521),
 ('흐리', 1.2179286027996072),
 ('긁어', 1.2097283121008695),
 ('입히는', 1.2026402081085654),
 ('모색한', 1.1763896256628235),
 ('김', 1.145379680059789),
 ('입', 1.1383539756396253),
 ('지워진', 1.1312750443916961),
 ('반복한', 1.1038214644730957),
 ('확대되', 1.0899180713515009),
 ('오려', 1.0711339026174966),
 ('마련되', 1.0690188180375684),
 ('아픈', 1.0658811840980642)]

다른 텍스트를 가져와 이를 기반으로 검색해보자 즉, tf-idf기반으로 가장 유사도가 높은 전시를 뽑아보자 

In [113]:
query = """빈센트 반 고흐의 대표작 중 하나로 꼽히는 <별이 빛나는 밤>은 그가 고갱과 다툰 뒤 자신의 귀를 자른 사건 이후 생레미의 요양원에 있을 때 그린 것이다. 반 고흐에게 밤하늘은 무한함을 표현하는 대상이었고, 이보다 먼저 제작된 아를의 <밤의 카페 테라스>나 <론 강 위로 별이 빛나는 밤>에서도 별이 반짝이는 밤의 정경을 다루었다. 반 고흐는 동생 테오에게 쓴 편지에서 “오늘 아침 나는 해가 뜨기 한참 전에 창문을 통해 아무것도 없고 아주 커 보이는 샛별밖에 없는 시골을 보았다.”고 했다. 이 샛별은 그림 가운데 왼쪽에 있는 커다란 흰 별일 것이다. 그가 그린 밤하늘에서는 구름과 대기, 별빛과 달빛이 폭발하고 있다. 황량하고 짙은 파란색 하늘은 세상의 종말을 연상케 하고, 그 위로는 구름이 소용돌이치며 떠있다. 달과 별의 둘레에는 뿌옇게 무리가 져있다.

비연속적이고 동적인 터치로 그려진 하늘은 굽이치는 두꺼운 붓놀림으로 불꽃같은 사이프러스와 연결되고, 그 아래의 마을은 대조적으로 평온하고 고요하다. 마을은 있는 그대로 그린 것이 아니라 부분적으로는 고안되었는데, 교회 첨탑은 반 고흐의 고향 네덜란드를 연상시킨다. 그는 병실 밖으로 내다보이는 밤 풍경을 기억과 상상을 결합시켜 그렸는데, 이는 자연에 대한 반 고흐의 내적이고 주관적인 표현을 구현하고 있다. 수직으로 높이 뻗어 땅과 하늘을 연결하는 사이프러스는 전통적으로 무덤이나 애도와 연관된 나무이지만, 반 고흐는 죽음을 불길하게 보지 않았다. 

그는 “별을 보는 것은 언제나 나를 꿈꾸게 한다.”면서 “왜 하늘의 빛나는 점들에는 프랑스 지도의 검은 점처럼 닿을 수 없을까? 타라스콩이나 루앙에 가려면 기차를 타듯이, 우리는 별에 다다르기 위해 죽는다.”고 했다. 이 시기에 그의 필치는 더욱 두꺼워지고 더욱 열정적으로 변했으며, 꿈틀거리는 듯한 선은 별의 광채를 한층 두드러지게 한다. 이 시기의 특징인 회오리치는 듯 꿈틀거리는 필치는 강렬한 색과 결합되어 감정을 더욱 격렬하게 표현한다. 이 곡선의 화필은 굽이치는 운동감을 표현하면서, 그림 전체를 율동적인 흐름으로 통합한다.
[네이버 지식백과] 별이 빛나는 밤 [The Starry Night] - 빈센트 반 고흐 (The Bridgeman Art Library, 지엔씨미디어)"""

ads 기준으로 해보자

In [22]:
tw = Twitter()

In [114]:
query_ads = {}
maxCount = 0


for w in [tag[0] for tag in tw.pos(query) if tag[1] in ['Adjective','Adverb','Verb','Exclamation']]:
    if w in query_ads.keys():
        query_ads[w] += 1
    else:
        query_ads[w] = 1

    if maxCount < query_ads[w]:
        maxCount = query_ads[w]

print(query_ads)

{'꼽히는': 1, '빛나는': 4, '다툰': 1, '자른': 1, '있': 1, '표현하는': 1, '이었': 1, '된': 2, '아': 2, '반짝이는': 1, '다루었': 1, '쓴': 1, '뜨': 1, '없고': 1, '커': 1, '보이는': 1, '없는': 1, '보았': 1, '했': 2, '있는': 2, '커다란': 1, '흰': 1, '있다': 2, '짙은': 1, '하고': 1, '떠': 1, '져': 1, '그려진': 1, '치는': 3, '두꺼운': 1, '같은': 1, '연결되': 1, '고요하다': 1, '아니': 1, '되었': 1, '시킨': 1, '내다보': 1, '시켜': 1, '그렸': 1, '이는': 1, '뻗어': 1, '연결하는': 1, '하게': 2, '않았': 1, '보는': 1, '언제나': 1, '나를': 1, '꿈꾸게': 1, '한': 3, '검은': 1, '닿': 1, '없': 1, '가려': 1, '타': 1, '다': 1, '다르': 1, '죽는': 1, '필': 2, '두꺼워': 1, '변했': 1, '두드러지': 1, '강렬한': 1, '되어': 1, '표현한': 1, '표현하면': 1}


In [103]:
# # noun 기준 코드
# query_nouns = {}
# maxCount = 0

# for w in tw.nouns(query):
#     if w in query_nouns.keys():
#         query_nouns[w] += 1
#     else:
#         query_nouns[w] = 1

#     if maxCount < query_nouns[w]:
#         maxCount = query_nouns[w]
    
# print(query_nouns)

쿼리를 한 doc으로 보고 가중치를 구한다, 여기서는 tf만 구했다

In [122]:
query_weight = {}

for k,v in query_ads.items():
    query_weight[k] = K + (1-K) * (v/maxCount)
    print('{0} | {1} + {2} *({3}/{4})  =  {5}'.format(
            k, K, (1-K), v, maxCount, query_weight[k]
        ))

꼽히는 | 0.5 + 0.5 *(1/4)  =  0.625
빛나는 | 0.5 + 0.5 *(4/4)  =  1.0
다툰 | 0.5 + 0.5 *(1/4)  =  0.625
자른 | 0.5 + 0.5 *(1/4)  =  0.625
있 | 0.5 + 0.5 *(1/4)  =  0.625
표현하는 | 0.5 + 0.5 *(1/4)  =  0.625
이었 | 0.5 + 0.5 *(1/4)  =  0.625
된 | 0.5 + 0.5 *(2/4)  =  0.75
아 | 0.5 + 0.5 *(2/4)  =  0.75
반짝이는 | 0.5 + 0.5 *(1/4)  =  0.625
다루었 | 0.5 + 0.5 *(1/4)  =  0.625
쓴 | 0.5 + 0.5 *(1/4)  =  0.625
뜨 | 0.5 + 0.5 *(1/4)  =  0.625
없고 | 0.5 + 0.5 *(1/4)  =  0.625
커 | 0.5 + 0.5 *(1/4)  =  0.625
보이는 | 0.5 + 0.5 *(1/4)  =  0.625
없는 | 0.5 + 0.5 *(1/4)  =  0.625
보았 | 0.5 + 0.5 *(1/4)  =  0.625
했 | 0.5 + 0.5 *(2/4)  =  0.75
있는 | 0.5 + 0.5 *(2/4)  =  0.75
커다란 | 0.5 + 0.5 *(1/4)  =  0.625
흰 | 0.5 + 0.5 *(1/4)  =  0.625
있다 | 0.5 + 0.5 *(2/4)  =  0.75
짙은 | 0.5 + 0.5 *(1/4)  =  0.625
하고 | 0.5 + 0.5 *(1/4)  =  0.625
떠 | 0.5 + 0.5 *(1/4)  =  0.625
져 | 0.5 + 0.5 *(1/4)  =  0.625
그려진 | 0.5 + 0.5 *(1/4)  =  0.625
치는 | 0.5 + 0.5 *(3/4)  =  0.875
두꺼운 | 0.5 + 0.5 *(1/4)  =  0.625
같은 | 0.5 + 0.5 *(1/4)  =  0.625
연결되 | 0.5 + 0.

In [123]:
#문서별로 norm을 구한다
doc_len = {} #문서1:길이, 문서2:길이 ...

for k,v in TFIDF.items(): #k는 문서id, v는 단어를 키, 가중치를 밸류로 가진 딕셔너리
    sumPow = 0
    
    for t in v:
        sumPow += v[t]**2
    
    doc_len[k] = sqrt(sumPow)

In [124]:
#쿼리의 norm을 구한다
for k,v in query_weight.items(): #k는 문서id, v는 단어를 키, 가중치를 밸류로 가진 딕셔너리
#     print(k,v)
    sumPow += v**2
    
query_len = sqrt(sumPow)
query_len

14.714445993734014

In [125]:
result = {}
for k,v in query_weight.items():
    if k in invertedIdx.keys():
        for docid in invertedIdx[k]:
            if docid in result.keys():
                result[docid] += TFIDF[docid][k] * query_weight[k]
            else:
                result[docid] = TFIDF[docid][k] * query_weight[k]

In [126]:
for k,v in result.items():
    result[k] = v/(doc_len[k]*query_len) # 코사인 유사도

In [127]:
top = sorted(result.items(),reverse = True, key= lambda kv:kv[1])[:10] # 유사도 높은 전시 top 10

In [128]:
for k,v in top:
    print("[{0}] 유사도:{1}".format(k,v)) # 해당문서와 쿼리간의 정렬된 코사인 유사도를 도출

[3503] 유사도:0.05499725946706687
[5790] 유사도:0.051602620957071174
[12730] 유사도:0.05097031857804088
[5696] 유사도:0.04774590754843096
[14372] 유사도:0.04655719963517263
[7839] 유사도:0.045282252610838576
[1825] 유사도:0.04466985058089504
[10585] 유사도:0.04427984674649625
[2830] 유사도:0.04416506452655589
[15066] 유사도:0.044045524934394314


비슷하다고 뽑아놓은 전시...

http://www.arthub.co.kr/sub01/board03_view.htm?No=19337

빈도 기반의 유사도 측정은 한계가 있기에

W2V 결과에 TF-IDF값을 가중치로 곱하는 형태로 사용해보자

-> art2vec.ipynb 참고